In [4]:
import cv2
import json
import pandas as pd
from os import listdir
import re
import time
import numpy as np


def StandardScaler(x):
    '''数据标准化，且以21个骨骼点的第一个为原点'''
    x_array=np.array(x)
    x_array-=x_array[0]

    mean=np.mean(x_array)
    std=np.std(x_array)

    x_array-=mean
    x_array/=std

    x_corr=x_array.tolist()

    return x_corr


def Landmarks2array(hand_landmarks):
    '''将mediapipe的输出变为numpy.array形式(63, )'''
    # ->txt
    with open('temp/temp.txt','w') as f:
        print(hand_landmarks,file=f)

    # txt->json
    seq = re.compile(":")
    result = []

    with open('temp/temp.txt') as f:
        for line in f:
            lst = seq.split(line.strip())
            if (len(lst)>=2): 
                item = {str(lst[0]): float(lst[1])}
                result.append(item)    

    with open('temp/temp.json', 'w') as dump_f:
        json.dump(result,dump_f)
        
    # json->array
    with open('temp/temp.json') as f:
        data = json.load(f)
        x = []
        y = []
        z = []
        xyz=[]
        for pt in data:
            if 'x' in pt: x.append(pt['x'])
            if 'y' in pt: y.append(pt['y'])
            if 'z' in pt: z.append(pt['z'])
        
        # 数据标准化
        x=StandardScaler(x)
        y=StandardScaler(y)
        z=StandardScaler(z)

    xyz=x+y+z # 将21*3=63个值排成一列，便于下一步处理

    return np.array(xyz)
